# **HDSI Agrithon 2024**

### Team CodeCultivators

### Prompt 2 : Analyzing the relationship between farmer age demographics and agricultural production across different states in the contiguous United States.



In [ ]:
"!pip install --upgrade --quiet gcsfs google-cloud-storage


##Step 1 : Connecting to Drive

In [ ]:
from google.colab import auth
from google.colab import drive
import os

# Authenticate the user to access Google Drive
auth.authenticate_user()

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Verify that the drive is mounted
if os.path.exists('/content/drive'):
    print("Drive mounted successfully!")
else:
    print("Drive mounting failed!")

Mounted at /content/drive
Drive mounted successfully!


## Step 2 : Setting the directory path

In [ ]:
directory_path = '/content/drive/MyDrive/HDSI_Data'

## Step 3 : Initializing the cloud storage client

In [ ]:
## Initialize the Google Cloud Storage client.
from google.cloud import storage
client = storage.Client(project='HDSI-Agri-Datathon-2024')

In [ ]:
bucket_name = 'hdsi-agri-prompt-data'

## Access the specified bucket
bucket = client.bucket(bucket_name)

## List all blobs (files and folders) in the bucket
blobs = list(bucket.list_blobs())

## Initialize variables to count and store folder names and file types
folders = set()
file_types = set()
blob_count = 0

## Loop through all blobs to gather folder names and file types
for blob in blobs:
    blob_name = blob.name

    ## Check if it's a folder (by convention, ends with '/')
    if blob_name.endswith('/'):
        folders.add(blob_name)
    else:
        ## Capture file type
        file_extension = blob_name.split('.')[-1] if '.' in blob_name else 'Unknown'
        file_types.add(file_extension)

    blob_count += 1

## list the folders
print("Folders in the bucket:")
for folder in folders:
    print(f" - {folder}")

## Show the file types of the first 5 files (skip folders)
print("\nFirst 5 file types:")
counter = 0
for blob in blobs:
    if not blob.name.endswith('/'): #and counter < 5:
        file_name = blob.name
        file_extension = file_name.split('.')[-1] if '.' in file_name else 'Unknown'
        print(f"File name: {file_name}, File type: {file_extension}")
        counter += 1

## Total count of blobs
print(f"\nTotal number of blobs in the bucket: {blob_count}")

## Print the unique file types found
print(f"\nFile types in the bucket: {', '.join(file_types)}")


In [ ]:
from google.cloud import storage
import os

# Initialize the storage client
client = storage.Client()

# Specify the bucket name and folder
bucket_name = 'hdsi-agri-prompt-data'
prompt2_folder = 'HDSI_AGRI_Prompt_2/'  # Adjust if necessary
local_save_directory = '/content/drive/MyDrive/HDSI_Data'  # Local directory to save the files

# Access the specified bucket
bucket = client.bucket(bucket_name)

# List of files to download
files_to_download = [
    'HDSI_AGRI_Prompt_2/guide_prompt2_prompt3_sales.xlsx',
    'HDSI_AGRI_Prompt_2/inflation_GDP_price_index.xlsx',
    'HDSI_AGRI_Prompt_2/prompt2_demos_landtotals_county.csv',
    'HDSI_AGRI_Prompt_2/prompt2_prompt3_sales.csv',
    'HDSI_AGRI_Prompt_2/state_level_2002_2007_2012_2017_2022.xlsx',
    'HDSI_AGRI_Prompt_2/sales_data_county.csv'
]

# Create local directory if it doesn't exist
if not os.path.exists(local_save_directory):
    os.makedirs(local_save_directory)

# Download each specified file
for file_path in files_to_download:
    file_name = os.path.basename(file_path)  # Get the file name from the path
    local_file_path = os.path.join(local_save_directory, file_name)

    # Fetch the blob (file) from the bucket
    blob = bucket.blob(file_path)

    # Download the file to the local directory
    print(f"Downloading {file_path} to {local_file_path}...")
    blob.download_to_filename(local_file_path)

print(f"All specified files have been downloaded to {local_save_directory}.")


All specified files have been downloaded to /content/drive/MyDrive/HDSI_Data.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

## Step 4 : Processing the Data

## Dataset 1: Sales Data County

The dataset contains sales data for various agricultural products, including the number of operations and sales measured in dollars across different counties. It covers a wide range of crops like berries, soybeans, tobacco, vegetables, and wheat. Some columns contain missing or placeholder data, indicating unavailable values for certain products or regions.

### Loading the data

In [ ]:
# Load the data from the CSV file
file_path = '/content/drive/MyDrive/HDSI_Data/sales_data_county.csv'
sales_county_data = pd.read_csv(file_path)

# Check the first few rows to understand the structure of the data
print("Initial data:")
print(sales_county_data.head())

Initial data:
   STATE_FIPS_CODE  COUNTY_CODE  YEAR  \
0               28           83  2022   
1               36           93  2022   
2               13           79  2022   
3               29          127  2022   
4                8            1  2022   

  ANIMAL_TOTALS_INCL_PRODUCTS_OPERATIONS_WITH_SALES  \
0                                                21   
1                                                41   
2                                                86   
3                                               202   
4                                               345   

  ANIMAL_TOTALS_INCL_PRODUCTS_SALES_MEASURED_IN_DOLLARS  \
0                                         81,403,000      
1                                          2,055,000      
2                                         85,593,000      
3                                         31,588,000      
4                                         25,004,000      

  AQUACULTURE_TOTALS_OPERATIONS_WITH_SALES_DISTRIBUTI

### Filtering only those with county code " 999 "

999 is not a county number. Instead, it is the total of all county data (including disclosure values) for each year of the census (third column). The total of all the county data during a census year would not always equal the number shown in these "999" state census year rows when added together.

**The 999 code is important to us as we are doing a state-wise analysis, rather than a county-wise analysis**




In [ ]:
# Filter the data to keep only rows where COUNTY_CODE is 999
sales_county_data = sales_county_data[sales_county_data['COUNTY_CODE'] == 999]



1. Disclosure values marked as (D) were handled by replacing them with the respective state and year mean to retain year and state-level insights. Removing these values would have resulted in a significant loss of data for the analysis, so we opted to impute the state level mean instead.

2. Similarly, (Z) values, representing negligible percentages, were replaced with 0. This approach ensures data completeness while avoiding the elimination of rows or introducing skewness by imputing the mean for near-zero values.

3. All empty strings are also replaced groups by the State FIPS code and replaced by the state mean

In [ ]:
# Convert all columns that contain numbers but are formatted as strings, and handle symbols like commas
numeric_cols = sales_county_data.columns[sales_county_data.dtypes == 'object']
sales_county_data[numeric_cols] = sales_county_data[numeric_cols].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',', '').str.replace('[(D)]', '', regex=True), errors='coerce'))

# Replace NA values with the mean for each STATE_FIPS_CODE for numeric columns
for col in numeric_cols:
    if sales_county_data[col].dtype == 'float64' or sales_county_data[col].dtype == 'int64':
        # Fill NaN with state mean
        sales_county_data[col] = sales_county_data.groupby('STATE_FIPS_CODE')[col].transform(lambda x: x.fillna(x.mean()))
        # If still NaN, fill with overall mean
        sales_county_data[col].fillna(sales_county_data[col].mean(), inplace=True)

sales_county_data.head()


,STATE_FIPS_CODE,COUNTY_CODE,YEAR,ANIMAL_TOTALS_INCL_PRODUCTS_OPERATIONS_WITH_SALES,ANIMAL_TOTALS_INCL_PRODUCTS_SALES_MEASURED_IN_DOLLARS,AQUACULTURE_TOTALS_OPERATIONS_WITH_SALES_DISTRIBUTION,AQUACULTURE_TOTALS_SALES_DISTRIBUTION_MEASURED_IN_DOLLARS,BARLEY_OPERATIONS_WITH_SALES,BARLEY_SALES_MEASURED_IN_DOLLARS,BERRY_TOTALS_OPERATIONS_WITH_SALES,...,SOYBEANS_OPERATIONS_WITH_SALES,SOYBEANS_SALES_MEASURED_IN_DOLLARS,SPECIALTY_ANIMAL_TOTALS_EXCL_EQUINE_OPERATIONS_WITH_SALES,SPECIALTY_ANIMAL_TOTALS_EXCL_EQUINE_SALES_MEASURED_IN_DOLLARS,TOBACCO_OPERATIONS_WITH_SALES,TOBACCO_SALES_MEASURED_IN_DOLLARS,VEGETABLE_TOTALS_INCL_SEEDS_TRANSPLANTS_IN_THE_OPEN_OPERATIONS_WITH_SALES,VEGETABLE_TOTALS_INCL_SEEDS_TRANSPLANTS_IN_THE_OPEN_SALES_MEASURED_IN_DOLLARS,WHEAT_OPERATIONS_WITH_SALES,WHEAT_SALES_MEASURED_IN_DOLLARS
3078,30,999,2022,11368,2061081000,27.0,5208000.0,1510.000000,2.041710e+08,126.0,...,64.0,6.017000e+06,556.0,38229000.0,1794.477419,1.267902e+08,423.0,81850000.0,3898.0,1.248771e+09
3079,28,999,2022,14757,4929592000,162.0,261458000.0,566.292727,3.761093e+07,473.0,...,2926.0,1.720581e+09,577.0,15726000.0,7.000000,4.960000e+05,1303.0,128077000.0,355.0,3.757600e+07
3080,40,999,2022,45070,6952396000,37.0,4822000.0,21.000000,1.785000e+06,317.0,...,1110.0,7.708700e+07,1178.0,17029000.0,1794.477419,1.267902e+08,519.0,30981000.0,5136.0,5.674750e+08
3081,50,999,2022,2762,706712000,14.0,2143000.0,3.000000,7.700000e+04,457.0,...,44.0,2.598000e+06,404.0,5967000.0,1794.477419,1.267902e+08,752.0,40562000.0,15.0,4.990000e+05
3082,27,999,2022,22770,11338329000,98.0,19763000.0,336.000000,2.767300e+07,489.0,...,25334.0,5.087514e+09,1405.0,58149000.0,1794.477419,1.267902e+08,2648.0,446099000.0,3668.0,7.355040e+08


In [ ]:
sales_county_data_cleaned = sales_county_data

## Dataset 2: Land Totals county demos

The dataset contains detailed agricultural and land use data across counties, focusing on the number of operations and land utilization by different ownership types (e.g., full owner, part owner).

Here, the columns which have the operators and producers age are important for our analysis. It also includes information on producer tenure, with columns tracking the number of years producers have been on their present operation. The columns have mixed data types, and certain values appear to be missing or not applicable

In [ ]:
land_totals_data = pd.read_csv('/content/drive/MyDrive/HDSI_Data/prompt2_demos_landtotals_county.csv')
land_totals_data.head()


,STATE_FIPS_CODE,COUNTY_CODE,YEAR,AG_LAND_CROPLAND_HARVESTED_ACRES,AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_FULL_OWNER,AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_PART_OWNER,AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_TENANT,AG_LAND_CROPLAND_HARVESTED_NUMBER_OF_OPERATIONS,AG_LAND_CROPLAND_HARVESTED_NUMBER_OF_OPERATIONS_WHERE_TENURE_FULL_OWNER,AG_LAND_CROPLAND_HARVESTED_NUMBER_OF_OPERATIONS_WHERE_TENURE_PART_OWNER,...,PRODUCERS_WHITE_NUMBER_OF_PRODUCERS,PRODUCERS_YEARS_ON_ANY_OPERATION_6_TO_10_YEARS_NUMBER_OF_PRODUCERS,PRODUCERS_YEARS_ON_ANY_OPERATION_AVG_MEASURED_IN_YEARS,PRODUCERS_YEARS_ON_ANY_OPERATION_GE_11_YEARS_NUMBER_OF_PRODUCERS,PRODUCERS_YEARS_ON_ANY_OPERATION_LT_6_YEARS_NUMBER_OF_PRODUCERS,PRODUCERS_YEARS_ON_PRESENT_OPERATION_3_TO_4_YEARS_NUMBER_OF_PRODUCERS,PRODUCERS_YEARS_ON_PRESENT_OPERATION_5_TO_9_YEARS_NUMBER_OF_PRODUCERS,PRODUCERS_YEARS_ON_PRESENT_OPERATION_AVG_MEASURED_IN_YEARS,PRODUCERS_YEARS_ON_PRESENT_OPERATION_GE_10_YEARS_NUMBER_OF_PRODUCERS,PRODUCERS_YEARS_ON_PRESENT_OPERATION_LT_3_YEARS_NUMBER_OF_PRODUCERS
0,1,1,1997,27857,5834,13764,8259,246.0,132.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2002,26132,8897,13344,3891,201.0,113.0,76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,2007,18548,4298,8820,5430,196.0,100.0,77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,2012,29676,5607,16671,7398,204.0,102.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,2017,27557,4321,17027,6209,167.0,74.0,82.0,...,493.0,84.0,22.2,386.0,98.0,66.0,104.0,20.0,368.0,30.0


Selecting only the columns that have age

In [ ]:
# Create a subset of the dataframe with the specified columns
columns_to_consider = [
    'STATE_FIPS_CODE',
    'COUNTY_CODE',
    'YEAR',

    'OPERATORS_PRINCIPAL_AGE_25_TO_34_NUMBER_OF_OPERATORS',
    'OPERATORS_PRINCIPAL_AGE_35_TO_44_NUMBER_OF_OPERATORS',
    'OPERATORS_PRINCIPAL_AGE_45_TO_54_NUMBER_OF_OPERATORS',
    'OPERATORS_PRINCIPAL_AGE_55_TO_59_NUMBER_OF_OPERATORS',
    'OPERATORS_PRINCIPAL_AGE_60_TO_64_NUMBER_OF_OPERATORS',
    'OPERATORS_PRINCIPAL_AGE_65_TO_69_NUMBER_OF_OPERATORS',
    'OPERATORS_PRINCIPAL_AGE_GE_70_NUMBER_OF_OPERATORS',
    'OPERATORS_PRINCIPAL_AGE_LT_25_NUMBER_OF_OPERATORS',
    'OPERATORS_PRINCIPAL_AGE_AVG_MEASURED_IN_YEARS',
    'OPERATORS_ALL_NUMBER_OF_OPERATORS',

    'PRODUCERS_AGE_25_TO_34_NUMBER_OF_PRODUCERS',
    'PRODUCERS_AGE_35_TO_44_NUMBER_OF_PRODUCERS',
    'PRODUCERS_AGE_45_TO_54_NUMBER_OF_PRODUCERS',
    'PRODUCERS_AGE_55_TO_64_NUMBER_OF_PRODUCERS',
    'PRODUCERS_AGE_65_TO_74_NUMBER_OF_PRODUCERS',
    'PRODUCERS_AGE_GE_75_NUMBER_OF_PRODUCERS',
    'PRODUCERS_AGE_LT_25_NUMBER_OF_PRODUCERS',
    'PRODUCERS_AGE_AVG_MEASURED_IN_YEARS',
    'PRODUCERS_ALL_NUMBER_OF_PRODUCERS',

    'AG_LAND_CROPLAND_HARVESTED_ACRES',
    'AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_FULL_OWNER',
    'AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_PART_OWNER',
    'AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_TENANT',
    'AG_LAND_OWNED_IN_FARMS_ACRES',
    'AG_LAND_RENTED_FROM_OTHERS_IN_FARMS_ACRES',
    'FARM_OPERATIONS_ACRES_OPERATED',
    'FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_FULL_OWNER',
    'FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_PART_OWNER',
    'FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_TENANT'
]

# Extract the relevant subset of the dataframe
land_totals_data_cleaned = land_totals_data[columns_to_consider]


land_totals_data_cleaned.head()

,STATE_FIPS_CODE,COUNTY_CODE,YEAR,OPERATORS_PRINCIPAL_AGE_25_TO_34_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_35_TO_44_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_45_TO_54_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_55_TO_59_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_60_TO_64_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_65_TO_69_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_GE_70_NUMBER_OF_OPERATORS,...,AG_LAND_CROPLAND_HARVESTED_ACRES,AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_FULL_OWNER,AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_PART_OWNER,AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_TENANT,AG_LAND_OWNED_IN_FARMS_ACRES,AG_LAND_RENTED_FROM_OTHERS_IN_FARMS_ACRES,FARM_OPERATIONS_ACRES_OPERATED,FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_FULL_OWNER,FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_PART_OWNER,FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_TENANT
0,1,1,1997,24.0,60.0,91.0,60.0,66.0,37.0,92.0,...,27857,5834,13764,8259,NaN,NaN,112221,56357,38256,17608
1,1,1,2002,13.0,43.0,97.0,47.0,48.0,53.0,72.0,...,26132,8897,13344,3891,NaN,NaN,118468,71250,40182,7036
2,1,1,2007,27.0,59.0,77.0,56.0,70.0,36.0,84.0,...,18548,4298,8820,5430,NaN,NaN,110464,52595,36351,21518
3,1,1,2012,20.0,42.0,65.0,36.0,68.0,56.0,90.0,...,29676,5607,16671,7398,NaN,NaN,111504,55866,39819,15819
4,1,1,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27557,4321,17027,6209,NaN,NaN,113236,50119,50443,12674


####**Handling operators and producers**

In our analysis, we have generalized the term operator to include all producers across all years, including 2017 and 2022. This is because, for our purposes, we are not focusing on the expanded demographic data that includes additional individuals like family members who are involved in farm decision-making. By treating all producers as a an operator or a single stakeholder in the family, we maintain consistency in our dataset, simplifying the analysis by focusing only on those directly involved in operational activities.

### Data Manipulation
1. Creating new, more generalized age group columns by aggregating specific age ranges into buckets. This ensures consistency of ages between the producers group and the operators group.

2. Missing values (NaN) are replaced with 0 using .fillna(0) to ensure they do not interfere with the summation.

In [ ]:
# Sum the two columns, ignoring NA values, and create a new column
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_55_TO_64_NUMBER_OF_OPERATORS'] = (
    land_totals_data['OPERATORS_PRINCIPAL_AGE_55_TO_59_NUMBER_OF_OPERATORS'].fillna(0) +
    land_totals_data['OPERATORS_PRINCIPAL_AGE_60_TO_64_NUMBER_OF_OPERATORS'].fillna(0)
)

# Sum the two columns, ignoring NA values, and create a new column
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_GT_65_NUMBER_OF_OPERATORS'] = (
    land_totals_data['OPERATORS_PRINCIPAL_AGE_65_TO_69_NUMBER_OF_OPERATORS'].fillna(0) +
    land_totals_data['OPERATORS_PRINCIPAL_AGE_GE_70_NUMBER_OF_OPERATORS'].fillna(0)
)


land_totals_data_cleaned['PRODUCERS_AGE_GE_65_NUMBER_OF_PRODUCERS'] = (
    land_totals_data['PRODUCERS_AGE_65_TO_74_NUMBER_OF_PRODUCERS'].fillna(0) +
    land_totals_data['PRODUCERS_AGE_GE_75_NUMBER_OF_PRODUCERS'].fillna(0)
)

3. Dropping repetitive columns

In [ ]:
land_totals_data_cleaned.drop(columns = ['OPERATORS_PRINCIPAL_AGE_55_TO_59_NUMBER_OF_OPERATORS', 'OPERATORS_PRINCIPAL_AGE_60_TO_64_NUMBER_OF_OPERATORS',
'OPERATORS_PRINCIPAL_AGE_65_TO_69_NUMBER_OF_OPERATORS', 'OPERATORS_PRINCIPAL_AGE_GE_70_NUMBER_OF_OPERATORS',
                   'PRODUCERS_AGE_65_TO_74_NUMBER_OF_PRODUCERS', 'PRODUCERS_AGE_GE_75_NUMBER_OF_PRODUCERS'],inplace=True)

In [ ]:
land_totals_data_cleaned.head()

,STATE_FIPS_CODE,COUNTY_CODE,YEAR,OPERATORS_PRINCIPAL_AGE_25_TO_34_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_35_TO_44_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_45_TO_54_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_LT_25_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_AVG_MEASURED_IN_YEARS,OPERATORS_ALL_NUMBER_OF_OPERATORS,PRODUCERS_AGE_25_TO_34_NUMBER_OF_PRODUCERS,...,AG_LAND_CROPLAND_HARVESTED_ACRES_ON_OPERATIONS_WITH_TENURE_TENANT,AG_LAND_OWNED_IN_FARMS_ACRES,AG_LAND_RENTED_FROM_OTHERS_IN_FARMS_ACRES,FARM_OPERATIONS_ACRES_OPERATED,FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_FULL_OWNER,FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_PART_OWNER,FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_TENANT,OPERATORS_PRINCIPAL_AGE_55_TO_64_NUMBER_OF_OPERATORS,OPERATORS_PRINCIPAL_AGE_GT_65_NUMBER_OF_OPERATORS,PRODUCERS_AGE_GE_65_NUMBER_OF_PRODUCERS
0,1,1,1997,24.0,60.0,91.0,NaN,57.4,NaN,NaN,...,8259,NaN,NaN,112221,56357,38256,17608,126.0,129.0,0.0
1,1,1,2002,13.0,43.0,97.0,NaN,58.1,523,NaN,...,3891,NaN,NaN,118468,71250,40182,7036,95.0,125.0,0.0
2,1,1,2007,27.0,59.0,77.0,6.0,56.2,594,NaN,...,5430,NaN,NaN,110464,52595,36351,21518,126.0,120.0,0.0
3,1,1,2012,20.0,42.0,65.0,12.0,58.2,563,NaN,...,7398,NaN,NaN,111504,55866,39819,15819,104.0,146.0,0.0
4,1,1,2017,NaN,NaN,NaN,NaN,NaN,NaN,54.0,...,6209,NaN,NaN,113236,50119,50443,12674,0.0,0.0,206.0


### Merging the producer and operator column with each age category

To ensure consistency of age groups across, similar age groups of producers and operators are being merged to create a unified column for operators who fall into the same age group.

We are considering the primary stakeholder of the producers as the operator for our analysis.


**The 5 age groups are :**
  1. Age < 25
  2. Age 25 to 35
  3. Age 35 to 44
  4. Age 45 to 54
  5. Age 55 to 64
  6. Age >= 65


In [ ]:
# Merging the two columns to create a unified column for all years
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_25_TO_34_NUMBER_OF_OPERATORS'] = land_totals_data_cleaned[
    ['OPERATORS_PRINCIPAL_AGE_25_TO_34_NUMBER_OF_OPERATORS', 'PRODUCERS_AGE_25_TO_34_NUMBER_OF_PRODUCERS']
].bfill(axis=1).iloc[:,0]

# Dropping the PRODUCERS_AGE_25_TO_34_NUMBER_OF_PRODUCERS column as it is no longer needed
land_totals_data_cleaned.drop(columns=['PRODUCERS_AGE_25_TO_34_NUMBER_OF_PRODUCERS'],inplace=True)

In [ ]:
# Merging the two columns to create a unified column for all years
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_35_TO_44_NUMBER_OF_OPERATORS'] = land_totals_data_cleaned[
    ['OPERATORS_PRINCIPAL_AGE_35_TO_44_NUMBER_OF_OPERATORS', 'PRODUCERS_AGE_35_TO_44_NUMBER_OF_PRODUCERS']
].bfill(axis=1).iloc[:, 0]


# Dropping the PRODUCERS_AGE_35_TO_44_NUMBER_OF_PRODUCERS column as it is no longer needed
land_totals_data_cleaned.drop(columns=['PRODUCERS_AGE_35_TO_44_NUMBER_OF_PRODUCERS'],inplace=True)

In [ ]:
# Merging the two columns to create a unified column for all years
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_55_TO_64_NUMBER_OF_OPERATORS'] = land_totals_data_cleaned[
    ['OPERATORS_PRINCIPAL_AGE_55_TO_64_NUMBER_OF_OPERATORS', 'PRODUCERS_AGE_55_TO_64_NUMBER_OF_PRODUCERS']
].bfill(axis=1).iloc[:, 0]


# Dropping the PRODUCERS_AGE_55_TO_64_NUMBER_OF_PRODUCERS column as it is no longer needed
land_totals_data_cleaned.drop(columns=['PRODUCERS_AGE_55_TO_64_NUMBER_OF_PRODUCERS'],inplace=True)

In [ ]:
# Merging the two columns to create a unified column for all years
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_45_TO_54_NUMBER_OF_OPERATORS'] = land_totals_data_cleaned[
    ['OPERATORS_PRINCIPAL_AGE_45_TO_54_NUMBER_OF_OPERATORS', 'PRODUCERS_AGE_45_TO_54_NUMBER_OF_PRODUCERS']
].bfill(axis=1).iloc[:, 0]


# Dropping the PRODUCERS_AGE_45_TO_54_NUMBER_OF_PRODUCERS column as it is no longer needed
land_totals_data_cleaned.drop(columns=['PRODUCERS_AGE_45_TO_54_NUMBER_OF_PRODUCERS'],inplace=True)

In [ ]:
# Merging the two columns to create a unified column for all years
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_GT_65_NUMBER_OF_OPERATORS'] = land_totals_data_cleaned[
    ['OPERATORS_PRINCIPAL_AGE_GT_65_NUMBER_OF_OPERATORS', 'PRODUCERS_AGE_GE_65_NUMBER_OF_PRODUCERS']
].bfill(axis=1).iloc[:, 0]


# Dropping the PRODUCERS_AGE_GE_65_NUMBER_OF_PRODUCERS column as it is no longer needed
land_totals_data_cleaned.drop(columns=['PRODUCERS_AGE_GE_65_NUMBER_OF_PRODUCERS'], inplace=True)


In [ ]:
# Merging the two columns to create a unified column for all years
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_LT_25_NUMBER_OF_OPERATORS'] = land_totals_data_cleaned[
    ['OPERATORS_PRINCIPAL_AGE_LT_25_NUMBER_OF_OPERATORS', 'PRODUCERS_AGE_LT_25_NUMBER_OF_PRODUCERS']
].bfill(axis=1).iloc[:, 0]


# Dropping the PRODUCERS_AGE_LT_25_NUMBER_OF_PRODUCERS column as it is no longer needed
land_totals_data_cleaned.drop(columns=['PRODUCERS_AGE_LT_25_NUMBER_OF_PRODUCERS'], inplace=True)


In [ ]:
# Merging the two columns to create a unified column for all years
land_totals_data_cleaned['OPERATORS_PRINCIPAL_AGE_AVG_MEASURED_IN_YEARS'] = land_totals_data_cleaned[
    ['OPERATORS_PRINCIPAL_AGE_AVG_MEASURED_IN_YEARS', 'PRODUCERS_AGE_AVG_MEASURED_IN_YEARS']
].bfill(axis=1).iloc[:, 0]


# Dropping the PRODUCERS_AGE_AVG_MEASURED_IN_YEARS column as it is no longer needed
land_totals_data_cleaned.drop(columns=['PRODUCERS_AGE_AVG_MEASURED_IN_YEARS'],inplace=True)

In [ ]:
# Merging the two columns to create a unified column for all years
land_totals_data_cleaned['OPERATORS_ALL_NUMBER_OF_OPERATORS'] = land_totals_data_cleaned[
    ['OPERATORS_ALL_NUMBER_OF_OPERATORS', 'PRODUCERS_ALL_NUMBER_OF_PRODUCERS']
].bfill(axis=1).iloc[:, 0]


# Dropping the PRODUCERS_ALL_NUMBER_OF_PRODUCERS column as it is no longer needed
land_totals_data_cleaned.drop(columns=['PRODUCERS_ALL_NUMBER_OF_PRODUCERS'],inplace=True)

Handling any missing values and filling it with 0 as NaN occuarance is Nil or very less

In [ ]:
land_totals_data_cleaned.fillna(0, inplace =True)

Just like the previous dataset, we are retaining only the 999 county code as it corresponds to the summation of all the information of counties in a given state. This is helpful as we are performing state-wise analysis.

In [ ]:
# retaining only 999 county code
land_totals_data_cleaned = land_totals_data_cleaned[land_totals_data_cleaned['COUNTY_CODE']==999]

### Dataset 3: State Levels Data

The dataset appears to contain state-level agricultural data from different years, detailing the number of farm operators in various age groups across different sales categories. The data spans multiple states, capturing trends related to farming demographics and operational characteristics over time

### Loading the dataset

In [ ]:
state_level_data = pd.read_excel('/content/drive/MyDrive/HDSI_Data/state_level_2002_2007_2012_2017_2022.xlsx')

### ANSI to State mapping

In [ ]:
# dictionary mapping of state ANSI (FIPS) codes to state names
ansi_to_state = {1: "Alabama",2: "Alaska",4: "Arizona",5: "Arkansas",6: "California",8: "Colorado",9: "Connecticut",10: "Delaware",11: "District of Columbia",12: "Florida",13: "Georgia",15:
                 "Hawaii",16: "Idaho",17: "Illinois",18: "Indiana",19: "Iowa",20: "Kansas",21: "Kentucky",22: "Louisiana",23: "Maine",24: "Maryland",25: "Massachusetts",26: "Michigan", 27:
                 "Minnesota",28: "Mississippi",29: "Missouri",30: "Montana",31: "Nebraska",32: "Nevada",33: "New Hampshire",34: "New Jersey",35: "New Mexico",36: "New York",37: "North Carolina",
                 38: "North Dakota",39: "Ohio",40: "Oklahoma",41: "Oregon",42: "Pennsylvania",44: "Rhode Island",45: "South Carolina",46: "South Dakota",47: "Tennessee", 48: "Texas", 49: "Utah",
                 50: "Vermont", 51: "Virginia", 53: "Washington", 54: "West Virginia", 55: "Wisconsin", 56: "Wyoming", 99: "United States"}


* To address missing state names in the GEO column, we are imputing the Null values by referencing a predefined standard mapping, which ensures that each missing entry is replaced with its corresponding state name accurately. This approach maintains data integrity and consistency across the dataset.

In [ ]:
# filling whereever GEO is missing using ansi_to_state dict
state_level_data['GEO'] = state_level_data['GEO'].fillna(state_level_data['State ANSI'].map(ansi_to_state))

# replacing "-" with 0 in all columns
state_level_data = state_level_data.replace('-', 0)

def process_data(df):
    # Define transformation rules
    transformations = [
        ([
            'OPERATORS, PRINCIPAL, AGE 55 TO 59 - NUMBER OF OPERATORS',
            'OPERATORS, PRINCIPAL, AGE 60 TO 64 - NUMBER OF OPERATORS'
        ], 'OPERATORS_PRINCIPAL_AGE_55_TO_64_NUMBER_OF_OPERATORS'),

        ([
            'OPERATORS, PRINCIPAL, AGE 65 TO 69 - NUMBER OF OPERATORS',
            'OPERATORS, PRINCIPAL, AGE GE 70 - NUMBER OF OPERATORS'
        ], 'OPERATORS_PRINCIPAL_AGE_GT_65_NUMBER_OF_OPERATORS'),

        ([
            'PRODUCERS, AGE 65 TO 74 - NUMBER OF PRODUCERS',
            'PRODUCERS, AGE GE 75 - NUMBER OF PRODUCERS'
        ], 'PRODUCER_PRINCIPAL_AGE_GT_65_NUMBER_OF_PRODUCERS')
    ]

    # Process each group defined by 'State ANSI' and 'Year'
    results = []
    for (state_ansi, year), group in df.groupby(['State ANSI', 'Year']):
        for old_names, new_name in transformations:
            rows_to_merge = group[group['Data Item'].isin(old_names)]

            if not rows_to_merge.empty:
                # Calculate the sum of the rows to merge
                summed_value = rows_to_merge['NOT SPECIFIED'].astype(int).sum()

                # Update the first row with the summed value
                first_index = rows_to_merge.index[0]
                group.at[first_index, 'NOT SPECIFIED'] = summed_value
                group.at[first_index, 'Data Item'] = new_name

                # Drop the remaining rows
                group = group.drop(rows_to_merge.index[1:])

        results.append(group)

    # Combine all the processed groups into a single DataFrame
    return pd.concat(results, ignore_index=True)


Data preprocessing steps involve
1. Normalizing "Data Item" names and specific fixes for ease of analysis
2. Consolidate age groups
3. Employing Kmeans to fill missing values


In [ ]:
# Apply the data processing
state_level_data = process_data(state_level_data)

def process_and_aggregate(df):
    # Step 1: Normalize 'Data Item' names to ensure uniformity
    # Initial cleanup and standardization
    df['Data Item'] = df['Data Item'].str.replace('PRODUCERS', 'OPERATORS')
    df['Data Item'] = df['Data Item'].str.replace('PRODUCER_', 'OPERATORS_')
    df['Data Item'] = df['Data Item'].str.replace(' - ', '_')
    df['Data Item'] = df['Data Item'].str.replace('NUMBER OF ', '_NUMBER_OF_')

    # Specific fixes for averaged age items
    df['Data Item'] = df['Data Item'].str.replace('_AGE_AVG_MEASURED_IN_YEARS_MEASURED IN YEARS', '_AGE_AVG_MEASURED_IN_YEARS')
    df['Data Item'] = df['Data Item'].str.replace('_MEASURED IN YEARS', '')

    # Convert all numeric columns to numeric type for proper summing
    numeric_cols = df.columns[df.columns.str.contains('FARM SALES') | df.columns.str.contains('NOT SPECIFIED')]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Step 2: Sum values for rows that have the same 'Data Item' name after renaming
    grouped = df.groupby(['State ANSI', 'Year', 'GEO', 'Data Item']).sum().reset_index()

    return grouped


# Apply the processing and aggregation function
state_level_data = process_and_aggregate(state_level_data)

def standardize_and_consolidate_data(df):
    # Normalize and standardize 'Data Item' names
    df['Data Item'] = df['Data Item'].str.replace('PRINCIPAL', '').replace(',', '').replace(' - ', ' ').replace('  ', ' ')
    df['Data Item'] = df['Data Item'].str.replace('AGE ', 'AGE_')
    df['Data Item'] = df['Data Item'].str.replace(' ', '_')

    # Consolidate specific age ranges to "45 TO 54"
    df['Data Item'] = df['Data Item'].str.replace('AGE_45_TO_49_NUMBER_OF_OPERATORS', 'AGE_45_TO_54_NUMBER_OF_OPERATORS')
    df['Data Item'] = df['Data Item'].str.replace('AGE_50_TO_54_NUMBER_OF_OPERATORS', 'AGE_45_TO_54_NUMBER_OF_OPERATORS')
    df['Data Item'] = df['Data Item'].str.replace('__', '_')  # Clean up any double underscores

    # Convert all numeric columns to numeric type for proper summing
    numeric_cols = df.columns[df.columns.str.contains('FARM SALES') | df.columns.str.contains('NOT SPECIFIED')]
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Group by necessary identifiers and sum up values to consolidate data for the new age range
    grouped = df.groupby(['State ANSI', 'Year', 'GEO', 'Data Item']).sum().reset_index()

    # Pivot table to widen the format, using the standardized names
    pivoted_df = df.pivot_table(index=['State ANSI', 'Year', 'GEO'],
                                columns='Data Item',
                                values=numeric_cols,
                                aggfunc='sum')

    # Flatten the multi-level columns created by pivot
    pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]

    # Reset index to turn the indices back into columns
    pivoted_df.reset_index(inplace=True)

    return pivoted_df

# Apply the data standardization and consolidation function
state_level_data = standardize_and_consolidate_data(state_level_data)


def fill_missing_with_kmeans(df, n_clusters=3):
    # Identify numeric columns (excluding the identifier columns like State ANSI, Year, GEO)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    # Handle NaNs temporarily for clustering
    imputer = SimpleImputer(strategy='mean')
    df_numeric = df[numeric_cols]
    df_numeric_imputed = imputer.fit_transform(df_numeric)

    # Standardize the data (important for k-means)
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df_numeric_imputed)

    # Apply K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    df['Cluster'] = kmeans.fit_predict(df_scaled)

    # Calculate cluster means and round to nearest integer
    df_numeric = pd.DataFrame(df_numeric_imputed, columns=numeric_cols)
    df_numeric['Cluster'] = df['Cluster']
    cluster_means = df_numeric.groupby('Cluster').mean().round(0)

    # Impute missing values based on cluster means
    for column in numeric_cols:
        df[column] = df.apply(
            lambda row: cluster_means.loc[row['Cluster'], column] if pd.isna(row[column]) else row[column],
            axis=1
        )

    # Drop the cluster column if no longer needed
    df.drop('Cluster', axis=1, inplace=True)

    return df


# Fill missing data
state_level_data_cleaned = fill_missing_with_kmeans(state_level_data)
state_level_data_cleaned.head()


,State ANSI,Year,GEO,"FARM SALES: (1,000 TO 2,499 $)_OPERATORS,_,_AGE_25_TO_34_NUMBER_OF_OPERATORS","FARM SALES: (1,000 TO 2,499 $)_OPERATORS,_,_AGE_35_TO_44_NUMBER_OF_OPERATORS","FARM SALES: (1,000 TO 2,499 $)_OPERATORS,_,_AGE_45_TO_49_NUMBER_OF_OPERATORS","FARM SALES: (1,000 TO 2,499 $)_OPERATORS,_,_AGE_50_TO_54_NUMBER_OF_OPERATORS","FARM SALES: (1,000 TO 2,499 $)_OPERATORS,_,_AGE_LT_25_NUMBER_OF_OPERATORS","FARM SALES: (1,000 TO 2,499 $)_OPERATORS,_AGE,_AVG,_MEASURED_IN_YEARS","FARM SALES: (1,000 TO 2,499 $)_OPERATORS,_AGE_25_TO_34_NUMBER_OF_OPERATORS",...,"NOT SPECIFIED_OPERATORS,_,_AGE_LT_25_NUMBER_OF_OPERATORS","NOT SPECIFIED_OPERATORS,_AGE,_AVG,_MEASURED_IN_YEARS","NOT SPECIFIED_OPERATORS,_AGE_25_TO_34_NUMBER_OF_OPERATORS","NOT SPECIFIED_OPERATORS,_AGE_35_TO_44_NUMBER_OF_OPERATORS","NOT SPECIFIED_OPERATORS,_AGE_45_TO_54_NUMBER_OF_OPERATORS","NOT SPECIFIED_OPERATORS,_AGE_55_TO_64_NUMBER_OF_OPERATORS","NOT SPECIFIED_OPERATORS,_AGE_LT_25_NUMBER_OF_OPERATORS","NOT SPECIFIED_OPERATORS_AGE,_AVG,_MEASURED_IN_YEARS",NOT SPECIFIED_OPERATORS_AGE_55_TO_64_NUMBER_OF_OPERATORS,NOT SPECIFIED_OPERATORS_AGE_GT_65_NUMBER_OF_OPERATORS
0,1,2002,Alabama,282.0,1057.0,751.0,888.0,49.0,55.3,674.0,...,252.0,56.6,7472.0,12879.0,17879.0,28264.0,1702.0,58.0,12163.0,12970.0
1,1,2007,Alabama,342.0,813.0,686.0,773.0,34.0,55.0,674.0,...,240.0,57.6,7472.0,12879.0,17879.0,28264.0,1702.0,58.0,13765.0,15279.0
2,1,2012,Alabama,250.0,482.0,371.0,597.0,46.0,57.1,674.0,...,231.0,59.3,7472.0,12879.0,17879.0,28264.0,1702.0,58.0,12379.0,15827.0
3,1,2017,Alabama,300.0,891.0,728.0,850.0,40.0,56.0,386.0,...,365.0,57.0,3866.0,6808.0,12362.0,17374.0,1076.0,58.0,15813.0,23256.0
4,1,2022,Alabama,300.0,891.0,728.0,850.0,40.0,56.0,430.0,...,365.0,57.0,3887.0,7157.0,10460.0,15481.0,1109.0,58.7,15813.0,24683.0


### Merging Datasets

### Merging sales_county_data_cleaned and land_totals_data_cleaned

An inner join is being performed, where the STATE_FIPS_CODE and YEAR are the common columns of the 2 tables.

In [ ]:
# Perform an inner join on the columns 'STATE_FIPS_CODE', 'COUNTY_CODE', and 'YEAR'
sales_landtotals_merged = pd.merge(sales_county_data_cleaned, land_totals_data_cleaned, left_on=['STATE_FIPS_CODE', 'YEAR'], right_on=['STATE_FIPS_CODE', 'YEAR'],how='inner')

### Merging sales_landtotals_merged and state_level_data_cleaned

An inner join is being performed, where the STATE_FIPS_CODE and YEAR are the common columns of the 2 tables.

In [ ]:
sales_landtotals_statelevel_merged = pd.merge(sales_landtotals_merged, state_level_data_cleaned, left_on=['STATE_FIPS_CODE', 'YEAR'], right_on=['State ANSI', 'Year'],how='inner')

All 3 datasets are now cleaned and merged

In [ ]:
sales_landtotals_statelevel_merged.shape

(255, 286)

### Inflation Adjustment

1. Adjusting Inflation using GDP price index data.

Inflation must be adjusted in GDP calculations to account for changes in the price level over time. Without adjusting for inflation, nominal GDP (which reflects the current value of goods and services) can give a misleading sense of growth, as price increases alone can inflate GDP figures. By adjusting for inflation, we calculate real GDP, which reflects the actual growth in the quantity of goods and services produced, allowing for a more accurate comparison of economic performance over different periods.

The formula to calculate real GDP from nominal GDP by adjusting for inflation is:

**Real GDP= (Nominal GDP × 100) /  GDP Deflator**


were:

a. Nominal GDP is the market value of all goods and services produced in an economy, not adjusted for inflation. This is the GDP price index for the base year.

b. GDP Deflator is an index that reflects the change in prices (inflation or deflation) for all goods and services in the economy relative to a base year.

**We have considered the base year to be 2022, where the GDP price index is 117.996**

**GDP Price index to Year mapping**

In [ ]:
# GDP Price index for the years in dataset
gdp_price_index = {
    1997: 69.337,
    2002: 75.5,
    2007: 86.352,
    2012: 93.176,
    2017: 100,
    2022: 117.996
}

inflation_factor_2022=117.996

In [ ]:
# remove the suffixes added from join using regex
sales_landtotals_statelevel_merged.columns = sales_landtotals_statelevel_merged.columns.str.replace(r'_x|_y', '', regex=True)

# create a column with gdp index value corresponding to year
sales_landtotals_statelevel_merged['gdp_price_index'] = sales_landtotals_statelevel_merged['YEAR'].map(gdp_price_index)

In [ ]:
# adjusting for inflation
for col in sales_landtotals_statelevel_merged.columns:
    if col.endswith('_DOLLARS'):
        sales_landtotals_statelevel_merged[col + '_INFLATED'] = (sales_landtotals_statelevel_merged[col] * inflation_factor_2022) / sales_landtotals_statelevel_merged['gdp_price_index']

In [ ]:
merged_data=sales_landtotals_statelevel_merged

In [ ]:
merged_data.shape

(255, 345)

In [ ]:
merged_data.head()

,STATE_FIPS_CODE,COUNTY_CODE,YEAR,ANIMAL_TOTALS_INCL_PRODUCTS_OPERATIONS_WITH_SALES,ANIMAL_TOTALS_INCL_PRODUCTS_SALES_MEASURED_IN_DOLLARS,AQUACULTURE_TOTALS_OPERATIONS_WITH_SALES_DISTRIBUTION,AQUACULTURE_TOTALS_SALES_DISTRIBUTION_MEASURED_IN_DOLLARS,BARLEY_OPERATIONS_WITH_SALES,BARLEY_SALES_MEASURED_IN_DOLLARS,BERRY_TOTALS_OPERATIONS_WITH_SALES,...,POULTRY_TOTALS_INCL_EGGS_SALES_MEASURED_IN_DOLLARS_INFLATED,RICE_SALES_MEASURED_IN_DOLLARS_INFLATED,SHEEP_GOATS_TOTALS_INCL_WOOL_MOHAIR_MILK_SALES_MEASURED_IN_DOLLARS_INFLATED,SHORT_TERM_WOODY_CROPS_SALES_MEASURED_IN_DOLLARS_INFLATED,SORGHUM_SALES_MEASURED_IN_DOLLARS_INFLATED,SOYBEANS_SALES_MEASURED_IN_DOLLARS_INFLATED,SPECIALTY_ANIMAL_TOTALS_EXCL_EQUINE_SALES_MEASURED_IN_DOLLARS_INFLATED,TOBACCO_SALES_MEASURED_IN_DOLLARS_INFLATED,VEGETABLE_TOTALS_INCL_SEEDS_TRANSPLANTS_IN_THE_OPEN_SALES_MEASURED_IN_DOLLARS_INFLATED,WHEAT_SALES_MEASURED_IN_DOLLARS_INFLATED
0,30,999,2022,11368,2061081000,27.0,5208000.0,1510.000000,2.041710e+08,126.0,...,4.358800e+07,3.646538e+08,36852000.0,6000.000000,1.210000e+05,6.017000e+06,38229000.0,1.267902e+08,81850000.0,1.248771e+09
1,28,999,2022,14757,4929592000,162.0,261458000.0,566.292727,3.761093e+07,473.0,...,4.150516e+09,1.075920e+08,3480000.0,751000.000000,1.407000e+06,1.720581e+09,15726000.0,4.960000e+05,128077000.0,3.757600e+07
2,40,999,2022,45070,6952396000,37.0,4822000.0,21.000000,1.785000e+06,317.0,...,1.591977e+09,3.646538e+08,21914000.0,633236.802413,4.083700e+07,7.708700e+07,17029000.0,1.267902e+08,30981000.0,5.674750e+08
3,50,999,2022,2762,706712000,14.0,2143000.0,3.000000,7.700000e+04,457.0,...,2.199000e+07,3.646538e+08,6480000.0,633236.802413,7.047099e+07,2.598000e+06,5967000.0,1.267902e+08,40562000.0,4.990000e+05
4,27,999,2022,22770,11338329000,98.0,19763000.0,336.000000,2.767300e+07,489.0,...,1.943207e+09,3.646538e+08,33457000.0,108000.000000,4.370000e+05,5.087514e+09,58149000.0,1.267902e+08,446099000.0,7.355040e+08


We are handling the occurance of (D) by imputing it with the mean of the column.

In [ ]:
# Display the columns mentioned by the user
columns_of_interest = [
    "FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_FULL_OWNER",
    "FARM_OPERATIONS_ACRES_OPERATED_ON_OPERATIONS_WITH_TENURE_TENANT"
]


#Check for occurrences of the value "D" in the two specified columns
occurrences_of_D = merged_data[columns_of_interest].apply(lambda x: x.str.contains('(D)', na=False))

# Convert the columns to numeric, setting errors='coerce' to handle non-numeric values like "D"
merged_data[columns_of_interest] = merged_data[columns_of_interest].apply(pd.to_numeric, errors='coerce')

# Calculate the mean of each column, ignoring NaN values
means = merged_data[columns_of_interest].mean()

# Replace NaN values (previously "D") with the mean of the respective column
merged_data[columns_of_interest] = merged_data[columns_of_interest].fillna(means)

In [ ]:
# merged_data save it to csv
merged_data.to_csv('merged_data.csv', index=False)

* The data from 1997 is not published in a manner consistent with the methodology used for other years in the age-by-sales groups, leading to missing data.
* To address this, the 1997 data was handled by adjusting the values, either by adding or subtracting the difference observed between 2007 and 2002 for the corresponding columns.
*This approach ensures consistency across the dataset while accounting for the missing data from 1997.

In [ ]:
# Load the data
file_path = '/content/merged_data.csv'
data = pd.read_csv(file_path)

# Filter data for years 2002 and 2007 to calculate the differences
data_2002 = data[data['YEAR'] == 2002]
data_2007 = data[data['YEAR'] == 2007]

# Merge 2002 and 2007 data on state and county codes to align them for difference calculation
data_combined = pd.merge(data_2007, data_2002, on=['STATE_FIPS_CODE', 'COUNTY_CODE'], suffixes=('_2007', '_2002'))

# Calculate the differences for numeric columns
numeric_columns = [col for col in data.columns if data[col].dtype in ['int64', 'float64'] and col not in ['YEAR', 'STATE_FIPS_CODE', 'COUNTY_CODE']]
for col in numeric_columns:
    data_combined[col + '_diff'] = data_combined[col + '_2007'] - data_combined[col + '_2002']

# Prepare base 1997 data by copying data from any year, adjusting values, and setting the year to 1997
base_1997 = data[data['YEAR'] == 2002].copy()
base_1997['YEAR'] = 1997

for col in numeric_columns:
    if col + '_diff' in data_combined.columns:
        diff_series = data_combined.set_index(['STATE_FIPS_CODE', 'COUNTY_CODE'])[col + '_diff']
        base_1997.set_index(['STATE_FIPS_CODE', 'COUNTY_CODE'], inplace=True)
        base_1997[col] -= diff_series
        base_1997.reset_index(inplace=True)

# Append the modified 1997 data back to the original dataset
data = pd.concat([data, base_1997], ignore_index=True)
data.sort_values(by=['STATE_FIPS_CODE', 'COUNTY_CODE', 'YEAR'], inplace=True)





* Saving the final merged data after inflation adjustment

In [ ]:
# Save or return the modified dataset
data.to_csv('/content/Final_merged_data.csv',index=False)